In [26]:
import pandas as pd
import numpy as np

In [27]:
# Chargement du dataset Kaggle
df_kaggle = pd.read_csv("../../Data/2025/atp_tennis_2025.csv")

# Conversion de la colonne de date au bon format
df_kaggle["Date"] = pd.to_datetime(df_kaggle["Date"], errors='coerce')

# Filtrage sur l'année 2025 uniquement
df_2025 = df_kaggle[df_kaggle["Date"].dt.year == 2025].copy()

print(f"{len(df_2025)} matches found for 2025")
df_2025.head()

958 matches found for 2025


,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
64181,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Nishioka Y.,Lehecka J.,Lehecka J.,69,28,776,1660,3.20,1.36,2-6 7-6 4-6
64182,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Jarry N.,Bonzi B.,Jarry N.,35,75,1370,730,2.63,1.50,6-2 6-3
64183,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Vukic A.,Dimitrov G.,Dimitrov G.,68,10,778,3350,4.50,1.20,2-6 6-7
64184,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Thompson J.,Michelsen A.,Thompson J.,26,41,1745,1245,1.67,2.20,7-5 6-3
64185,Brisbane International,2025-01-02,ATP250,Outdoor,Hard,2nd Round,3,Lajovic D.,Mensik J.,Mensik J.,81,48,710,1136,4.00,1.25,3-6 2-6


In [28]:
# Dataset Jeff Sackmann pour structure de référence
df_jeff = pd.read_csv("../../Data/atp/atp_matches_2024.csv")
df_jeff.head()


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2024-0339,Brisbane,Hard,32,A,20240101,300,105777,2.0,NaN,...,58.0,44.0,16.0,11.0,8.0,9.0,14.0,2570.0,8.0,3660.0
1,2024-0339,Brisbane,Hard,32,A,20240101,299,208029,1.0,NaN,...,35.0,31.0,10.0,11.0,5.0,7.0,8.0,3660.0,39.0,1122.0
2,2024-0339,Brisbane,Hard,32,A,20240101,298,105777,2.0,NaN,...,39.0,24.0,14.0,10.0,5.0,7.0,14.0,2570.0,55.0,902.0
3,2024-0339,Brisbane,Hard,32,A,20240101,297,208029,1.0,NaN,...,51.0,31.0,16.0,10.0,3.0,5.0,8.0,3660.0,116.0,573.0
4,2024-0339,Brisbane,Hard,32,A,20240101,296,126128,NaN,NaN,...,37.0,27.0,16.0,10.0,5.0,8.0,39.0,1122.0,44.0,1021.0


In [29]:
# Colonnes à harmoniser
cols_jeff = set(df_jeff.columns)
cols_kaggle = set(df_2025.columns)

missing_in_kaggle = cols_jeff - cols_kaggle
extra_in_kaggle = cols_kaggle - cols_jeff

print("Colonnes manquantes dans Kaggle :", missing_in_kaggle)
print("Colonnes supplémentaires dans Kaggle :", extra_in_kaggle)


print("Colonnes dans jeff :", cols_jeff)
print("Colonnes dans Kaggle :", cols_kaggle)

Colonnes manquantes dans Kaggle : {'w_bpFaced', 'winner_id', 'l_ace', 'loser_rank', 'best_of', 'minutes', 'winner_ht', 'w_df', 'loser_entry', 'round', 'winner_rank', 'loser_name', 'l_2ndWon', 'draw_size', 'surface', 'loser_seed', 'winner_ioc', 'winner_hand', 'w_bpSaved', 'w_1stWon', 'w_SvGms', 'tourney_level', 'score', 'winner_seed', 'loser_ht', 'loser_ioc', 'loser_rank_points', 'loser_age', 'l_df', 'loser_hand', 'l_svpt', 'winner_age', 'winner_rank_points', 'l_SvGms', 'winner_entry', 'w_svpt', 'tourney_id', 'winner_name', 'loser_id', 'l_bpSaved', 'tourney_date', 'w_ace', 'w_1stIn', 'w_2ndWon', 'match_num', 'l_1stIn', 'l_1stWon', 'tourney_name', 'l_bpFaced'}
Colonnes supplémentaires dans Kaggle : {'Best of', 'Player_1', 'Series', 'Pts_2', 'Rank_1', 'Odd_1', 'Player_2', 'Winner', 'Court', 'Rank_2', 'Pts_1', 'Odd_2', 'Round', 'Surface', 'Score', 'Tournament', 'Date'}
Colonnes dans jeff : {'w_bpFaced', 'winner_id', 'l_ace', 'loser_rank', 'best_of', 'minutes', 'winner_ht', 'w_df', 'loser_e

In [30]:
# 1. Dictionnaire de renommage (Kaggle → Jeff)
rename_cols = {
    "Player_1": "winner_name",
    "Player_2": "loser_name",
    "Rank_1": "winner_rank",
    "Rank_2": "loser_rank",
    "Pts_1": "winner_rank_points",
    "Pts_2": "loser_rank_points",
    "Surface": "surface",
    "Date": "tourney_date",
    "Score": "score",
    "Round": "round",
    "Best of": "best_of",
    "Tournament": "tourney_name"
}

# 2. Appliquer le renommage
df_2025_renamed = df_2025.rename(columns=rename_cols)

# 3. Supprimer les colonnes inutiles de Kaggle (non présentes dans Jeff)
columns_to_drop = set(df_2025_renamed.columns) - set(df_jeff.columns)
df_2025_cleaned = df_2025_renamed.drop(columns=columns_to_drop)

# 4. Ajouter les colonnes manquantes de Jeff (avec NaN ou 0 selon le type)
for col in df_jeff.columns:
    if col not in df_2025_cleaned.columns:
        dtype = df_jeff[col].dtype
        default = 0 if np.issubdtype(dtype, np.number) else np.nan
        df_2025_cleaned[col] = default

# 5. Réordonner les colonnes pour matcher exactement Jeff
df_2025_cleaned = df_2025_cleaned[df_jeff.columns]

# Résultat final
print("Harmonisation terminée.")
print(f"Final shape: {df_2025_cleaned.shape}")
df_2025_cleaned.head(3)


Harmonisation terminée.
Final shape: (958, 49)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
64181,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,69,776,28,1660
64182,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,35,1370,75,730
64183,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,68,778,10,3350


In [31]:
col = ["winner_name", "winner_id"]

df_2025_cleaned[col]


,winner_name,winner_id
64181,Nishioka Y.,0
64182,Jarry N.,0
64183,Vukic A.,0
64184,Thompson J.,0
64185,Lajovic D.,0
...,...,...
65146,Draper J.,0
65147,Diallo G.,0
65148,Ruud C.,0
65149,Musetti L.,0


In [ ]:
import unicodedata
import pandas as pd

# --- 1. Helpers ---

def normalize_name(s):
    """Remove accents, spaces, punctuation, convert to lowercase ASCII."""
    if pd.isna(s):
        return ""
    s = str(s).lower().replace("-", "").replace(".", "").replace(" ", "")
    s = unicodedata.normalize('NFD', s).encode('ascii', 'ignore').decode("utf-8")
    return s.strip()

# --- 2. Load and prepare ATP player reference ---

players = pd.read_csv("../../Data/players/atp_players.csv", dtype={"player_id": int})

# Build normalized full names for fuzzy matching
players["name_full"] = (
    players["name_last"].fillna("").str.strip().str.title() + " " +
    players["name_first"].fillna("").str.strip().str.title()
)
players["name_norm"] = (
    players["name_last"].fillna('') + players["name_first"].fillna('').str[0]
).apply(normalize_name)

# Build short name format "Last F." for first pass
players["short_name"] = (
    players["name_last"].str.strip().str.title() + " " +
    players["name_first"].str.strip().str[0].str.upper()
)

# Dicts for exact and fallback
name_to_id = dict(zip(players["short_name"], players["player_id"]))
norm_to_id = dict(zip(players["name_norm"], players["player_id"]))

# --- 3. Clean 2025 dataset names ---

df_2025_cleaned["winner_last"] = df_2025_cleaned["winner_name"].str.extract(r"^([\w\-']+)")
df_2025_cleaned["winner_initial"] = df_2025_cleaned["winner_name"].str.extract(r"\s+(\w)")
df_2025_cleaned["loser_last"] = df_2025_cleaned["loser_name"].str.extract(r"^([\w\-']+)")
df_2025_cleaned["loser_initial"] = df_2025_cleaned["loser_name"].str.extract(r"\s+(\w)")

df_2025_cleaned["winner_norm"] = (
    df_2025_cleaned["winner_last"].fillna('') + df_2025_cleaned["winner_initial"].fillna('')
).apply(normalize_name)

df_2025_cleaned["loser_norm"] = (
    df_2025_cleaned["loser_last"].fillna('') + df_2025_cleaned["loser_initial"].fillna('')
).apply(normalize_name)

# --- 4. Define fallback fuzzy matching function ---

def fuzzy_find_id(norm, players_df):
    """Try to match based on normalized name being in full name."""
    for _, row in players_df.iterrows():
        if norm in normalize_name(row["name_full"]):
            return row["player_id"]
    return pd.NA

# --- 5. Apply ID resolution with fallback logic ---

# First pass (from earlier mappings if present)
df_2025_cleaned["winner_id"] = df_2025_cleaned["winner_norm"].map(norm_to_id)
df_2025_cleaned["loser_id"]  = df_2025_cleaned["loser_norm"].map(norm_to_id)

# Fallback: fuzzy contains logic only where still missing
df_2025_cleaned["winner_id"] = df_2025_cleaned.apply(
    lambda row: row["winner_id"] if pd.notna(row["winner_id"])
    else fuzzy_find_id(row["winner_norm"], players),
    axis=1
)

df_2025_cleaned["loser_id"] = df_2025_cleaned.apply(
    lambda row: row["loser_id"] if pd.notna(row["loser_id"])
    else fuzzy_find_id(row["loser_norm"], players),
    axis=1
)

# Convert to nullable integer
df_2025_cleaned["winner_id"] = df_2025_cleaned["winner_id"].astype("Int64")
df_2025_cleaned["loser_id"] = df_2025_cleaned["loser_id"].astype("Int64")

# --- 6. Summary ---

missing = (df_2025_cleaned["winner_id"].isna() | df_2025_cleaned["loser_id"].isna()).sum()
total = len(df_2025_cleaned)

print(f"ID resolved: {total - missing} / {total}")
print(f"Still missing: {missing}")

if missing > 0:
    print("\nExamples of unmatched rows:")
    display(df_2025_cleaned[
        df_2025_cleaned["winner_id"].isna() | df_2025_cleaned["loser_id"].isna()
    ][["winner_name", "loser_name"]].head(20))

# --- 7. Manual patch for known unresolved players ---

manual_fixes = {
    "Mpetshi G.": 208659  # Giovanni Mpetshi Perricard
}

for short_name, pid in manual_fixes.items():
    df_2025_cleaned.loc[
        (df_2025_cleaned["winner_name"] == short_name) & (df_2025_cleaned["winner_id"].isna()),
        "winner_id"
    ] = pid
    df_2025_cleaned.loc[
        (df_2025_cleaned["loser_name"] == short_name) & (df_2025_cleaned["loser_id"].isna()),
        "loser_id"
    ] = pid


# Drop temporary columns
df_2025_cleaned.drop(columns=[
    "winner_last", "winner_initial", "loser_last", "loser_initial",
    "winner_norm", "loser_norm"
], inplace=True)


# Final summary
missing = (df_2025_cleaned["winner_id"].isna() | df_2025_cleaned["loser_id"].isna()).sum()
print(f"\nFinal ID resolution: {len(df_2025_cleaned) - missing} / {len(df_2025_cleaned)}")

/tmp/ipykernel_187533/352802210.py:16: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("../../Data/players/atp_players.csv", dtype={"player_id": int})


ID resolved: 947 / 958
Still missing: 11

🔍 Examples of unmatched rows:


,winner_name,loser_name
64187,Tiafoe F.,Mpetshi G.
64190,Mpetshi G.,Mensik J.
64192,Opelka R.,Mpetshi G.
64313,Mpetshi G.,Monfils G.
64638,Mpetshi G.,Zhang Zh.
64653,Mpetshi G.,Medvedev D.
64752,Griekspoor T.,Mpetshi G.
64833,Mpetshi G.,Thompson J.
64953,Mpetshi G.,Thompson J.
65008,Medjedovic H.,Mpetshi G.



Final ID resolution: 958 / 958
